# Apply a Simple Bag-of-Words Approach

[![Open In Colab](colab-badge.svg)](https://colab.research.google.com/github/alexisperrier/intro2nlp/blob/master/notebooks/intro2nlp_06_07_vectorizers.ipynb)



In [ ]:
# install spacy if you haven't done so already and download the small English model
!pip install -U spacy
!python -m spacy download en_core_web_sm

# install scikit learn
!pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
    '2 cups of flour',
    'replace the flour',
    'replace the keyboard in 2 minutes',
    'do you prefer Windows or Mac',
    'the Mac has the most noisy keyboard',
]
X = vectorizer.fit_transform(corpus)
X.todense()

# Classification

The simplified brown corpus is available in the data folder of this github repository


In [ ]:
import pandas as pd
df = pd.read_csv("../data/brown_corpus_extract_humor_science_fiction.csv")
df.shape

In [ ]:
df.head()

## Load spacy 


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

Define a function that 

- removes stopwords
- removes punctuation signs
- lemmatizes the text


In [ ]:
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_punct)]
    return ' '.join(tokens)


In [ ]:
text = '''These are the good times; Leave your cares behind'''
print(preprocess(text))

Add a couple of punctuation signs with multi charaters as stopwords to the ```nlp``` spacy model

In [ ]:
nlp.Defaults.stop_words.add("`,")
nlp.Defaults.stop_words.add("``")

Let's apply our preprocess function to the simplified Brown Corpus

In [ ]:
df['processed_text'] = df.text.apply(lambda txt : preprocess(txt))

short text with just a few tokens won't have enough information for the classification model that we want to train. Let's add a count of the number of tokens for each text and filter out the text with less than N tokens.

In [ ]:
df['token_count'] = df.processed_text.apply(lambda txt : len(txt.split())  )

In [ ]:
df.token_count.describe()

In [ ]:
df.token_count.hist(bins = 100)

Let's remove texts that have less than 4 tokens

In [ ]:
df = df[df.token_count > 4]
df.reset_index(inplace = True, drop = True)

In [ ]:
df[df.token_count > 4].topic.value_counts()

## Vectorize the Data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df.processed_text)

X is now a sparse matrix of 1258 rows by 4749 columns

In [ ]:
X.shape

Transform the topic from string to integer

In [ ]:

df.loc[df.topic == 'humor', 'topic' ] = 0
df.loc[df.topic == 'science_fiction', 'topic' ] = 1
df.topic = df.topic.astype(int)
# define the target variable
y = df.topic


In [ ]:
y

In [ ]:
X

## Train a classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# 1. Declare the model
clf = MultinomialNB()

# 2. Train the model
clf.fit(X, y)

# 3. Make predictions 
yhat = clf.predict(X)

# 4. score
print("Accuracy: ",accuracy_score(y, yhat))

To really train a classifier, we need to split the data in two parts: a training subset on which we train the model and a test set on which we evaluate the model. 

The test set simulates data that the model has not seen during its training and gives us a way to measure how the model extrapolates on unseen data.

We need to split the data after we've vectorized the text, otherwise sont tokens may be present in the test set but not in the training set. 

To split the data into a train and a test set we use scikit's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.



In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)




In [ ]:
print(X_train.shape)
print(X_test.shape)

Let's train the same model, this time on the train set and evaluate it on the test set



In [ ]:
# 1. Declare the model
clf = MultinomialNB()

# 2. Train the model
clf.fit(X_train, y_train)

# 3. Make predictions  on test set
yhat = clf.predict(X_test)

# 4. score
print("Accuracy: ",accuracy_score(y_test, yhat))

The accuracy is down to 77.4% which makes more sense than the outstanding 98% we previously obtained.

# tf-idf

Let's now compare the model performance when we use a tf-idf vectorization approach instead of a simple tf / count vectorizer.

The code is similar and we use scikit's default parameter for [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(df.processed_text)
y = df.topic

# split test train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# train and evaluate the model
# 1. Declare the model
clf = MultinomialNB()

# 2. Train the model
clf.fit(X_train, y_train)

# 3. Make predictions  on test set
yhat = clf.predict(X_test)

# 4. score
print("Accuracy: ",accuracy_score(y_test, yhat))

In this particular case, not much difference between the 2 vectorizers. 

# term to term matrix

Here is the code to generate the term to term matrix in chapter 7




In [ ]:
from nltk.tokenize import word_tokenize
from itertools import combinations
from collections import Counter

# The corpus
sentences = ['ways to replace the noisy Mac keyboard',
'do you prefer Windows or Mac',
'the Mac has a noisy keyboard',
'ways to install Windows on a Mac',
'you need a Windows license to install Windows on a Mac'
]

# vocab without stopwords
vocab = sorted(list(set(word_tokenize(' '.join(sentences)))))
stopwords = ['a', 'do', 'has', 'in', 'on', 'or', 'the', 'to', 'you', 'your']
vocab = sorted([tk.lower() for tk in vocab if tk not in stopwords])

print('Vocabulary:\n',vocab,'\n')

# tokenize
token_sent_list = [word_tokenize(sen) for sen in sentences]
print('Each sentence in token form:\n',token_sent_list,'\n')

# co occurrence window
k=3

# Definitely not an elegant way to create the term to term matrix
co_occ = {ii:Counter({jj:0 for jj in vocab if jj!=ii}) for ii in vocab}


for sen in token_sent_list:
    sen = [tk.lower() for tk in sen if tk not in stopwords]
    for ii in range(len(sen)):
        if ii < k:
            c = Counter(sen[0:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        elif ii > len(sen)-(k+1):
            c = Counter(sen[ii-k::])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        else:
            c = Counter(sen[ii-k:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c

# Having final matrix in dict form lets you convert it to different python data structures
co_occ = {ii:dict(co_occ[ii]) for ii in vocab}

# convert to DataFrame

df = pd.DataFrame(co_occ)
df.fillna(0, inplace = True)
for c in df.columns:
    df[c] = df[c].astype(int)

# here goes
df
